# Star Predection

The dataset yelp_business_official_test_empty.csv contains 8 new businesses that are not
found in dataset provided by Yelp. These 8 businesses are missing their star rating. Using what
we’ve learned in this course, you are to “predict” the stars for these 8 new businesses.

For example, if a new business was a fast food restaurant in Austin, TX, you might “predict” the
star rating by assigning the average rating for similar restaurants in Austin, TX.

In [1]:
import pandas as pd
import numpy as np

# Introduction
The dataset that I used to predict the star rating for new businesses is 'yelp_academic_dataset_business.csv' file.

# 1. Load the Datasets

In [2]:
# Import the 'yelp_dataset/yelp_business_official_test_empty.csv' file and set the 'business_id' as its index.
business_test_empty = pd.read_csv('yelp_dataset/yelp_business_official_test_empty.csv', 
                                  index_col = 'business_id')
business_test_empty

,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
business_id,,,,,,,,,,,,
1,"""Zingerman's Delicatessen""",NaN,"""422 Detroit St""",Ann Arbor,MI,48104,42.284682,-83.745071,NaN,1754,1,Delis;Breakfast & Brunch;Sandwiches;Restaurants
2,"""A & R Auto Care""",NaN,"""1202 N Cannon Blvd""",Kannapolis,NC,28083,35.510807,-80.608472,NaN,1,1,Automotive;Towing
3,"""Starbucks""",NaN,"""1135 Washington Blvd""",Ogden,UT,84404,41.245215,-111.970461,NaN,21,1,Food;Coffee & Tea
4,"""Starbucks""",NaN,"""5210 S Cicero Ave""",Chicago,IL,60638,41.798023,-87.743579,NaN,2,1,Food;Coffee & Tea
5,"""Starbucks""",NaN,"""4200 Conroy Rd""",Orlando,FL,32839,28.485466,-81.432003,NaN,30,1,Food;Coffee & Tea
6,"""The Tin Fox""",NaN,"""2616 Monroe St""",Madison,WI,53711,43.057715,-89.428370,NaN,7,1,American (New);Restaurants;Coffee & Tea;Food;N...
7,"""Working Draft Beer Company""",NaN,"""1129 E Wilson St""",Madison,WI,53703,43.083359,-89.365438,NaN,24,1,Food;Breweries
8,"""Il Covo""",NaN,"""585 College Street""",Toronto,ON,M6G 1B2,43.655166,-79.413312,NaN,21,1,Restaurants;Italian
9,"""Hawaii Nails & Spa""",NaN,"""1642 Bloor Street W""",Toronto,ON,M6P 1A7,43.655774,-79.456633,NaN,4,1,Beauty & Spas;Day Spas;Nail Salons


In [3]:
# Import the 'yelp_dataset/yelp_academic_dataset_business.csv' file and name it business
business = pd.read_csv('yelp_dataset/yelp_academic_dataset_business.csv')
business.head()

,address,attributes,attributes.AcceptsInsurance,attributes.AgesAllowed,attributes.Alcohol,attributes.Ambience,attributes.BYOB,attributes.BYOBCorkage,attributes.BestNights,attributes.BikeParking,...,hours.Wednesday,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,...,11:0-21:0,1,51.091813,-114.031675,Minhas Micro Brewery,NaN,T2E 6L6,24,4.0,AB
1,NaN,NaN,NaN,NaN,none,NaN,NaN,NaN,NaN,False,...,NaN,0,35.960734,-114.939821,CK'S BBQ & Catering,NaN,89002,3,4.5,NV
2,1335 rue Beaubien E,NaN,NaN,NaN,beer_and_wine,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,...,10:0-22:0,0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC
3,211 W Monroe St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,33.449999,-112.076979,Geico Insurance,NaN,85003,8,1.5,AZ
4,2005 Alyth Place SE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8:0-17:0,1,51.035591,-114.027366,Action Engine,NaN,T2H 0N5,4,2.0,AB


# 2. Prediction Process
First of all, I want to predict the star rating for these new businesses by the average star rating for businesses in the <b>same city</b>.

Take a close look at 'state' column of 'business_test_empty' dataframe, there're 7 businesses in the United States and 3 businesses in Canada. So for better prediction, I want to leave only the information with the correct US state abbreviations, plus the information about Toronto, ON, Canada. 

In [4]:
# Create a list of state abbreviations of the United States, as well as 'ON'(Canada)
states = ['AL', 'AK', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 
          'LA', 'ME', 'MD', 'MA', 'MI', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 
          'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'ON']
# Transform the list into a dataframe
df_state = pd.DataFrame({'state': states})
df_state.head()

,state
0,AL
1,AK
2,AR
3,AZ
4,CA


In [5]:
# I don't want to change the original dataset, so copy the original business dataset and assign it to 'df_business'
df_business = business.copy()

In [6]:
# Use 'pd.merge' to get a new dataframe that only contains the 
# information about the business with right state abbreviations that I need.
# Assign the new dataframe to 'new_df'
new_df = pd.merge(df_business, df_state, on = 'state')
new_df.head()

,address,attributes,attributes.AcceptsInsurance,attributes.AgesAllowed,attributes.Alcohol,attributes.Ambience,attributes.BYOB,attributes.BYOBCorkage,attributes.BestNights,attributes.BikeParking,...,hours.Wednesday,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,NaN,NaN,NaN,NaN,none,NaN,NaN,NaN,NaN,False,...,NaN,0,35.960734,-114.939821,CK'S BBQ & Catering,NaN,89002,3,4.5,NV
1,703 N Rancho Dr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,36.178348,-115.176916,Citi Trends,NaN,89106,4,4.0,NV
2,1549 N Rancho Dr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10:0-18:0,1,36.188386,-115.186124,Nevada Title And Payday Loans,NaN,89106,4,1.0,NV
3,"3940 Martin Luther King Blvd, Ste 101",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,...,11:0-18:0,0,36.192284,-115.159272,CakesbyToi,NaN,89106,3,1.5,NV
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,36.260816,-115.171130,Park Stone Pavers,NaN,89031,20,5.0,NV


In [7]:
# Group 'state' and 'city' together and get the average star rating of the businesses in a city.
df_groupby_state_city = new_df.groupby(['state', 'city'])['stars'].aggregate([np.mean]).rename(
    columns = {'mean': 'average star rating'})

df_groupby_state_city[:10]

average star rating
state city                                            
AL    Chandler                                3.000000
      Henderson Nevada                        5.000000
AR    Mesa                                    5.000000
      Phoenix                                 5.000000
AZ    Ahwahtukee                              5.000000
      Ahwatukee                               3.705882
      Ahwatukee Foothills Village             5.000000
      Anthem                                  3.557692
      Arrowhead                               3.500000
      Avondale                                3.496983

In [8]:
# Use pd.merge to get a new dataframe that merge the 'business_test_empty' and the 'df_groupby_state_city'
# together, so that the resulting dataframe will have a 'average star rating' as a column. Assign the new 
# dataframe to 'predict_star'
predict_star = pd.merge(business_test_empty, df_groupby_state_city, 
                         left_on = ['state', 'city'], right_index = True, 
                         how = 'left')
predict_star

,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,average star rating
business_id,,,,,,,,,,,,,
1,"""Zingerman's Delicatessen""",NaN,"""422 Detroit St""",Ann Arbor,MI,48104,42.284682,-83.745071,NaN,1754,1,Delis;Breakfast & Brunch;Sandwiches;Restaurants,NaN
2,"""A & R Auto Care""",NaN,"""1202 N Cannon Blvd""",Kannapolis,NC,28083,35.510807,-80.608472,NaN,1,1,Automotive;Towing,3.416185
3,"""Starbucks""",NaN,"""1135 Washington Blvd""",Ogden,UT,84404,41.245215,-111.970461,NaN,21,1,Food;Coffee & Tea,NaN
4,"""Starbucks""",NaN,"""5210 S Cicero Ave""",Chicago,IL,60638,41.798023,-87.743579,NaN,2,1,Food;Coffee & Tea,NaN
5,"""Starbucks""",NaN,"""4200 Conroy Rd""",Orlando,FL,32839,28.485466,-81.432003,NaN,30,1,Food;Coffee & Tea,NaN
6,"""The Tin Fox""",NaN,"""2616 Monroe St""",Madison,WI,53711,43.057715,-89.428370,NaN,7,1,American (New);Restaurants;Coffee & Tea;Food;N...,3.654173
7,"""Working Draft Beer Company""",NaN,"""1129 E Wilson St""",Madison,WI,53703,43.083359,-89.365438,NaN,24,1,Food;Breweries,3.654173
8,"""Il Covo""",NaN,"""585 College Street""",Toronto,ON,M6G 1B2,43.655166,-79.413312,NaN,21,1,Restaurants;Italian,3.490099
9,"""Hawaii Nails & Spa""",NaN,"""1642 Bloor Street W""",Toronto,ON,M6P 1A7,43.655774,-79.456633,NaN,4,1,Beauty & Spas;Day Spas;Nail Salons,3.490099


After the first operation, 6 businesses received predicted star rating while other 4 businesses didn't. The reason is that the <b>cities</b>(or <b>states</b>) where the 4 businesses are located failed to find the same match in the 'new_df' dataset.

So, let's do some more exploration.

In [9]:
missing_rating_states = ['MI', 'UT', 'IL', 'FL'] # The 4 states where the businesses didn't received predicted rating
states = ['AL', 'AK', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 
          'LA', 'ME', 'MD', 'MA', 'MI', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 
          'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'ON']

It is clear that 'IL' and 'FL' are in the 'new_df' dataset while 'MI' and 'UT' are not. 

What about cities?

In [10]:
'Chicago' in new_df[new_df['state'] == 'IL']['city']

False

In [11]:
'Orlando' in new_df[new_df['state'] == 'FL']['city']

False

No matching cities in 'new_df' dataset. So I decided to predict the star rating for businesses in these two cities by the average rating of their corresponding states.

Let's start the second operation!

In [12]:
# Group the new_df by 'state' and get the average star rating of the businesses in state.
df_groupby_state_IL_FL = new_df.groupby('state').aggregate(
    {'stars': np.mean}).loc[['IL', 'FL']].rename(columns = {'stars': 'average star rating'})

df_groupby_state_IL_FL

,average star rating
state,
IL,3.510842
FL,4.000000


In [13]:
# Fill the results obtained above into 'predict_star'
predict_star.iloc[3, 12] = 3.510842
predict_star.iloc[4, 12] = 4.000000

In [14]:
predict_star

,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,average star rating
business_id,,,,,,,,,,,,,
1,"""Zingerman's Delicatessen""",NaN,"""422 Detroit St""",Ann Arbor,MI,48104,42.284682,-83.745071,NaN,1754,1,Delis;Breakfast & Brunch;Sandwiches;Restaurants,NaN
2,"""A & R Auto Care""",NaN,"""1202 N Cannon Blvd""",Kannapolis,NC,28083,35.510807,-80.608472,NaN,1,1,Automotive;Towing,3.416185
3,"""Starbucks""",NaN,"""1135 Washington Blvd""",Ogden,UT,84404,41.245215,-111.970461,NaN,21,1,Food;Coffee & Tea,NaN
4,"""Starbucks""",NaN,"""5210 S Cicero Ave""",Chicago,IL,60638,41.798023,-87.743579,NaN,2,1,Food;Coffee & Tea,3.510842
5,"""Starbucks""",NaN,"""4200 Conroy Rd""",Orlando,FL,32839,28.485466,-81.432003,NaN,30,1,Food;Coffee & Tea,4.000000
6,"""The Tin Fox""",NaN,"""2616 Monroe St""",Madison,WI,53711,43.057715,-89.428370,NaN,7,1,American (New);Restaurants;Coffee & Tea;Food;N...,3.654173
7,"""Working Draft Beer Company""",NaN,"""1129 E Wilson St""",Madison,WI,53703,43.083359,-89.365438,NaN,24,1,Food;Breweries,3.654173
8,"""Il Covo""",NaN,"""585 College Street""",Toronto,ON,M6G 1B2,43.655166,-79.413312,NaN,21,1,Restaurants;Italian,3.490099
9,"""Hawaii Nails & Spa""",NaN,"""1642 Bloor Street W""",Toronto,ON,M6P 1A7,43.655774,-79.456633,NaN,4,1,Beauty & Spas;Day Spas;Nail Salons,3.490099


Great! Now we only have 2 businesses that didn't receive predicted star rating. Since 'new_df' contains no information about their states, I have to think about another way for the prediction.

The method I decided to use is to first classify the business categories in 'new_df' dataset, and then group the dataset by different classified categories and calculate the average star rating for each group. Finally I can use these values to predict the remaining 2 businesses.

Now, let's start the third operation!

In [15]:
# Before digging into the 'categories', remove all the null values, and select only the columns that I need.
new_df_sub = new_df[new_df['categories'].notnull()][['categories', 'stars']]
new_df_sub.head()

,categories,stars
0,"Chicken Wings, Burgers, Caterers, Street Vendo...",4.5
1,"Shopping, Fashion, Department Stores",4.0
2,"Financial Services, Check Cashing/Pay-day Loan...",1.0
3,"American (Traditional), Food, Bakeries, Restau...",1.5
4,"Home Services, Masonry/Concrete, Professional ...",5.0


In order to classify the categories, I need to retrieve the key words for the 2 businesses' categories. Those key words will help to classify and screen the two types of businesses that I want to predict.

In [16]:
# Get the key words of the "Zingerman's Delicatessen" category
predict_star[predict_star['state'] == 'MI']['categories']

business_id
1    Delis;Breakfast & Brunch;Sandwiches;Restaurants
Name: categories, dtype: object

In [17]:
# Get the key words of the "Starbucks" category
predict_star[predict_star['state'] == 'UT']['categories']

business_id
3    Food;Coffee & Tea
Name: categories, dtype: object

Now, create a function to classify the categories.

In [18]:
# This function is used to classify the categories in 'new_df_sub' into 3 categories.
def classify(s):
    s = s.lower()
    category = ''
    if 'delis' in s or 'breakfast' in s or 'brunch' in s or 'sandwiche' in s or 'restaurant' in s:
        category = 'Zingerman_like'
    elif 'food' in s or 'coffee' in s or 'tea' in s:
        category = 'Starbucks_like'
    else:
        category = 'Else'
    return category

In [19]:
# Apply the function to 'categories' of 'new_df_sub' and create a new column called 'classified', 
# after this step, I will get the 'categories' of 'new_df_sub' being classified into 3 types.
new_df_sub['classified'] = new_df_sub.categories.apply(classify)
new_df_sub.head()

,categories,stars,classified
0,"Chicken Wings, Burgers, Caterers, Street Vendo...",4.5,Zingerman_like
1,"Shopping, Fashion, Department Stores",4.0,Else
2,"Financial Services, Check Cashing/Pay-day Loan...",1.0,Else
3,"American (Traditional), Food, Bakeries, Restau...",1.5,Zingerman_like
4,"Home Services, Masonry/Concrete, Professional ...",5.0,Else


In [20]:
# Group the new_df_sub by 'classified', and calculate the average star rating in each group
new_df_sub.groupby('classified').aggregate({'stars': np.mean}).rename(
    columns = {'stars': 'average star rating'})

,average star rating
classified,
Else,3.735955
Starbucks_like,3.666807
Zingerman_like,3.420675


In [21]:
# Fill the results obtained above into 'predict_star'
predict_star.iloc[0, 12] = 3.420675
predict_star.iloc[2, 12] = 3.666807

In [22]:
# Round the predicted star to 2 decimals.
predict_star['average star rating'] = np.round(predict_star['average star rating'], 3)

In [23]:
predict_star

,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,average star rating
business_id,,,,,,,,,,,,,
1,"""Zingerman's Delicatessen""",NaN,"""422 Detroit St""",Ann Arbor,MI,48104,42.284682,-83.745071,NaN,1754,1,Delis;Breakfast & Brunch;Sandwiches;Restaurants,3.421
2,"""A & R Auto Care""",NaN,"""1202 N Cannon Blvd""",Kannapolis,NC,28083,35.510807,-80.608472,NaN,1,1,Automotive;Towing,3.416
3,"""Starbucks""",NaN,"""1135 Washington Blvd""",Ogden,UT,84404,41.245215,-111.970461,NaN,21,1,Food;Coffee & Tea,3.667
4,"""Starbucks""",NaN,"""5210 S Cicero Ave""",Chicago,IL,60638,41.798023,-87.743579,NaN,2,1,Food;Coffee & Tea,3.511
5,"""Starbucks""",NaN,"""4200 Conroy Rd""",Orlando,FL,32839,28.485466,-81.432003,NaN,30,1,Food;Coffee & Tea,4.000
6,"""The Tin Fox""",NaN,"""2616 Monroe St""",Madison,WI,53711,43.057715,-89.428370,NaN,7,1,American (New);Restaurants;Coffee & Tea;Food;N...,3.654
7,"""Working Draft Beer Company""",NaN,"""1129 E Wilson St""",Madison,WI,53703,43.083359,-89.365438,NaN,24,1,Food;Breweries,3.654
8,"""Il Covo""",NaN,"""585 College Street""",Toronto,ON,M6G 1B2,43.655166,-79.413312,NaN,21,1,Restaurants;Italian,3.490
9,"""Hawaii Nails & Spa""",NaN,"""1642 Bloor Street W""",Toronto,ON,M6P 1A7,43.655774,-79.456633,NaN,4,1,Beauty & Spas;Day Spas;Nail Salons,3.490


The star prediction for all new businesses are now completed.